In [ ]:
%pip install -q faker
%pip install -q jsonschema

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
properties_list = [
    {"name": {"type": "string", "minLength": 1, "maxLength": 50}},
    {"organization": {"type": "string", "minLength": 1, "maxLength": 50}},
    {"job_title": {"type": "string", "minLength": 1, "maxLength": 50}},
    {"department": {"type": "string", "minLength": 1, "maxLength": 50}},
    {"salary": {"type": "number", "minimum": 0}},
    {"age": {"type": "integer", "minimum": 0, "maximum": 120}},
    {"email": {"type": "string", "format": "email"}},
    # {"address": {"type": "string", "minLength": 1, "maxLength": 100}},
    # {"phone": {"type": "string", "pattern": "^\\+?[0-9\\- ]{7,15}$"}},
    # {"gender": {"type": "string", "enum": ["male", "female", "other"]}},
    # {"date_of_birth": {"type": "string", "format": "date"}},
    # {"is_active": {"type": "boolean", "default": "True"}},
    # {"created_at": {"type": "string", "format": "date-time"}},
    # {"updated_at": {"type": "string", "format": "date-time"}},
    # {"country": {"type": "string", "minLength": 1, "maxLength": 50}},
    # {"city": {"type": "string", "minLength": 1, "maxLength": 50}},
    # {"postal_code": {"type": "string", "minLength": 1, "maxLength": 20}},
    # {"username": {"type": "string", "minLength": 1, "maxLength": 50}},
    # {"password": {"type": "string", "minLength": 8, "maxLength": 100}},
    # {"profile_picture": {"type": "string", "format": "uri", "minLength": 1, "maxLength": 200}},
    # {"bio": {"type": "string", "minLength": 1, "maxLength": 500}},
    # {"website": {"type": "string", "format": "uri", "minLength": 1, "maxLength": 100}},
    # {"skills": {"type": "array", "items": {"type": "string"}}},
    # {"preferences": {"type": "object", "additionalProperties": {"type": "string"}}},
    # # Medical and healthcare related properties
    # {"blood_type": {"type": "string", "enum": ["A+", "A-", "B+", "B-", "AB+", "AB-", "O+", "O-"]}},
    # {"height_cm": {"type": "number", "minimum": 0, "maximum": 180}},
    # {"weight_kg": {"type": "number", "minimum": 0, "maximum": 200}},
    # {"allergies": {"type": "array", "items": {"type": "string"}}},
    # {"medical_conditions": {"type": "array", "items": {"type": "string"}}},
    # {"medications": {"type": "array", "items": {"type": "string"}}},
    # {"emergency_contact": {"type": "object"}},
    # {"insurance_provider": {"type": "string"}},
    # {"policy_number": {"type": "string"}},
    # {"last_checkup_date": {"type": "string", "format": "date"}},
    # {"doctor_name": {"type": "string"}},    
    # {"smoker": {"type": "boolean"}},
    # {"alcohol_consumption": {"type": "string", "enum": ["none", "occasional", "regular"]}},
    # {"heart_rate": {"type": "integer", "minimum": 0}},
    # {"blood_pressure": {"type": "string", "pattern": "^(\\d{1,3})/(\\d{1,3})$"}},  # Format: systolic/diastolic
    # # Health insurance related fields
    # {"insurance_plan_type": {"type": "string", "enum": ["HMO", "PPO", "EPO", "POS", "Other"]}},
    # {"insurance_coverage_start": {"type": "string", "format": "date"}},
    # {"insurance_coverage_end": {"type": "string", "format": "date"}},
    # {"insurance_member_id": {"type": "string", "minLength": 1, "maxLength": 50}},
    # {"insurance_group_number": {"type": "string", "minLength": 1, "maxLength": 50}},
    # {"insurance_contact_number": {"type": "string", "pattern": "^\\+?[0-9\\- ]{7,15}$"}},
    # {"insurance_status": {"type": "string", "enum": ["active", "inactive", "pending"]}}
]

In [ ]:
import random
from faker import Faker

faker = Faker()

for prop in properties_list:
    type = prop.get("type", None)
    if "type" == "string":
        if "enum" in prop.keys():
            value = faker.random_element(elements=prop["enum"])            
        elif prop_schema.get("format") == "email":
            lambda: None if include_nulls and random.random() < self.null_threshold else self._faker.email()

        elif prop_schema.get("format") == "date":
            lambda: None if include_nulls and random.random() < self.null_threshold else self._faker.date(pattern="%y%m%d")
        
        elif prop_schema.get("format") == "time":
            lambda: None if include_nulls and random.random() < self.null_threshold else self._faker.time(pattern="%H%M")
